In [13]:
import pandas as pd
from transformers import AutoTokenizer
from huggingface_hub import InferenceClient
import os


In [14]:
HUGGINGFACEHUB_API_TOKEN = "YOUR_API_TOKEN"

In [15]:
from dataclasses import dataclass


@dataclass
class DatasetConfig:
    """Dataset config class."""

    name: str
    augmentation_system_prompt: str
    augmentation_task_prompt: str
    classification_system_prompt: str
    classification_task_prompt: str

def parse_output(input_string: str):
    input_list = input_string.split("\n")  # split the input by newline characters
    output_list = []
    for item in input_list:
        item = item.lstrip(
            "0123456789. "
        )  # remove enumeration and any leading whitespace
        if item:  # skip empty items
            output_list.append(item)
    return output_list

In [16]:
HF_HUB_MODELS = {
    "llama-2-70b": "meta-llama/Llama-2-70b-chat-hf",
    "llama-2-13b": "meta-llama/Llama-2-13b-chat-hf",
    "llama-2-7b": "meta-llama/Llama-2-7b-chat-hf",
}

CrowdflowerConfig = DatasetConfig(
    name="crowdflower",
    augmentation_system_prompt="You are an advanced AI writer. Your job is to help write examples of social media comments that convey certain emotions. Emotions to be considered are: sadness, enthusiasm, empty, neutral, worry, love, fun, hate, happiness, relief, boredom, surprise, anger.",
    augmentation_task_prompt="""The following social media text conveys the emotion {label}. Write 9 new semantically similar examples in the style of a social media comment, that show the same intent and emotion.""",
    classification_system_prompt="You are an advanced classifying AI. You are tasked with classifying the emotion of a text. The emotions are: sadness, enthusiasm, empty, neutral, worry, love, fun, hate, happiness, relief, boredom, surprise, anger.",
    classification_task_prompt="Based on the following social media text, classify the emotion of the text. You answer MUST only be one of the emotions. Your answer MUST be exactly one of ['sadness', 'enthusiasm', 'empty', 'neutral', 'worry', 'love', 'fun', 'hate', 'happiness', 'relief', 'boredom', 'surprise', 'anger']. The answer must be lowercased.",
)

In [5]:
class HuggingfaceChatTemplate:
    def __init__(self, model_name: str):
        self.model_name: str = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.use_default_system_prompt = False

    def get_template_classification(self, system_prompt: str, task: str) -> str:
        chat = [
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": """{task}\n{few_shot}\nText: {text}\nAnswer: """.format(
                    task=task,
                    few_shot="{few_shot}",
                    text="{text}",
                ),
            },
        ]

        return self.tokenizer.apply_chat_template(chat, tokenize=False)

    def get_template_augmentation(self, system_prompt: str, task: str) -> str:
        chat = [
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": """{task}\nText: {text}\nAnswer: """.format(
                    task=task,
                    text="{text}",
                ),
            },
        ]

        return self.tokenizer.apply_chat_template(chat, tokenize=False)




In [6]:
config = CrowdflowerConfig

In [6]:
llm = InferenceClient(
        model=HF_HUB_MODELS["llama-2-13b"],
        token=HUGGINGFACEHUB_API_TOKEN,
    )

In [15]:
template = HuggingfaceChatTemplate(
        model_name=HF_HUB_MODELS["llama-2-70b"],
    ).get_template_classification(
        system_prompt=config.classification_system_prompt,
        task=config.classification_task_prompt,
    )

In [ ]:


    

    train = pd.read_json(os.path.join(CROWDFLOWER_DATA_DIR, "train.json"))
    # few_shot_samples = few_shot_sampling(df=train, n=1, per_class_sampling=True)

    output = llm.text_generation(
        template.format(
            few_shot="",
            text="@meganmansyn Hahahaha! It's not horrible, if others were singing with I'm sure it could work. I wish I could afford my own drum set",
        ),
        max_new_tokens=25,
        temperature=0.9,
        repetition_penalty=1.2,
    )

    a = 1

# Simpler version

In [40]:
docs = """"Document [1](Title: List of Nobel laureates in Physics) The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad R\u00f6ntgen, of Germany, who received 150,782 SEK, which is equal to 7,731,004 SEK in December 2007.  John Bardeen is the only laureate to win the prize twice\u2014in 1956 and 1972. Maria Sk\u0142odowska-Curie also won two Nobel Prizes, for physics in 1903 and chemistry in 1911. William Lawrence Bragg was, until October 2014, the youngest ever Nobel laureate; he won the prize in 1915 at the age of 25. Two women have won the prize: Curie and Maria Goeppert-Mayer (1963). As of 2017, the prize has been awarded\nDocument [2](Title: Asian Americans in science and technology) Prize in physics for discovery of the subatomic particle J/\u03c8. Subrahmanyan Chandrasekhar shared the 1983 Nobel Prize in Physics and had the Chandra X-ray Observatory named after him. Steven Chu shared the 1997 Nobel Prize in Physics for his research in cooling and trapping atoms using laser light. Daniel Tsui shared the 1998 Nobel Prize in Physics in 1998 for helping discover the fractional Quantum Hall effect. Yoichiro Nambu received the 2008 Nobel Prize in Physics for his work on the consequences of spontaneously broken symmetries in field theories. In 2009, Charles K. Kao was awarded Nobel Prize in Physics\nDocument [3](Title: Scientist) and pursued through a unique method, was essentially in place. Ram\u00f3n y Cajal won the Nobel Prize in 1906 for his remarkable observations in neuroanatomy. Marie Curie became the first female to win the Nobel Prize and the first person to win it twice. Her efforts led to the development of nuclear energy and Radio therapy for the treatment of cancer. In 1922, she was appointed a member of the International Commission on Intellectual Co-operation by the Council of the League of Nations. She campaigned for scientist's right to patent their discoveries and inventions. She also campaigned for free access\nDocument [4](Title: Norwegian Americans) science, Ernest Lawrence won the Nobel Prize in Physics in 1939. Lars Onsager won the 1968 Nobel Prize in Chemistry. Norman Borlaug, father of the Green Revolution, won the Nobel Peace Prize in 1970. Christian B. Anfinsen won the Nobel Prize for chemistry in 1972. Ivar Giaever won the Nobel Prize in Physics 1973. Carl Richard Hagen is noted for his work in physics. In engineering, Clayton Jacobson II is credited with the invention of the modern personal watercraft. Ole Singstad was a pioneer of underwater tunnels. Ole Evinrude invented the first outboard motor with practical commercial application, recognizable today\nDocument [5](Title: Maria Goeppert Mayer) of US postage stamps, along with Melvin Calvin, Asa Gray, and Severo Ochoa. Her papers are in the Geisel Library at the University of California, San Diego, and the university's physics department is housed in Mayer Hall, which is named after her and her husband. Maria Goeppert Mayer Maria Goeppert Mayer (June 28, 1906 \u2013 February 20, 1972) was a German-born American theoretical physicist, and Nobel laureate in Physics for proposing the nuclear shell model of the atomic nucleus. She was the second woman to win a Nobel Prize in physics, the first being Marie Curie. A graduate of the\nDocument [6](Title: University of Chicago) Medal, which is rewarded annually to the best economist under the age of 40, has also been awarded to 4 current members of the university faculty. Notable faculty in physics have included the speed of light calculator A. A. Michelson, elementary charge calculator Robert A. Millikan, discoverer of the Compton Effect Arthur H. Compton, the creator of the first nuclear reactor Enrico Fermi, \"the father of the hydrogen bomb\" Edward Teller, \"one of the most brilliant and productive experimental physicists of the twentieth century\" Luis Walter Alvarez, Murray Gell-Mann who introduced the quark, second female Nobel laureate Maria Goeppert-Mayer, the\nDocument [7](Title: E\u0301cole normale supe\u0301rieure (Paris)) was also awarded the Abel prize. In addition, eight \"normaliens\" have gone on to receive the Nobel Prize in Physics: Claude Cohen-Tannoudji, Pierre-Gilles de Gennes, Albert Fert, Alfred Kastler, Gabriel Lippmann, Louis N\u00e9el, Jean Baptiste Perrin and Serge Haroche, while other ENS physicists include such major figures as Paul Langevin, famous for developing Langevin dynamics and the Langevin equation. Alumnus Paul Sabatier won the Nobel Prize in Chemistry. A ranking of universities worldwide based on ratios of alumni to Nobel prize-winners published in 2016 by American scholars Stephen Hsu and Jonathan Wai placed ENS as the first university worldwide, far\nDocument [8](Title: Marie Curie) named in her honor. Marie Curie Marie Sk\u0142odowska Curie (; ; ; born Maria Salomea Sk\u0142odowska; 7 November 18674 July 1934) was a Polish and naturalized-French physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize, the first person and only woman to win twice, the only person to win a Nobel Prize in two different sciences, and was part of the Curie family legacy of five Nobel Prizes. She was also the first woman to become a professor at the University of Paris, and in 1995 became the first woman\nDocument [9](Title: St John's College, Cambridge) Nobel Prize winners: Sir Edward Appleton, for discovering the Appleton layer, Sir John Cockcroft KCB, physicist who first split the atom, Allan Cormack, for the invention of the CAT scan, Paul Dirac, one of the founders of quantum mechanics, Sir Nevill Francis Mott, for work on the behaviour of electrons in magnetic solids, Abdus Salam, for unifying the electromagnetic force and the weak force, Frederick Sanger, molecular biologist, Maurice Wilkins, awarded Nobel prize for Medicine or Physiology with Watson and Crick for discovering the structure of DNA, and Eric Maskin (visiting & honorary fellow), awarded the Nobel Memorial Prize in\nDocument [10](Title: Arthur H. Rosenfeld) Nobel Prize-winning Italian physicist. Rosenfeld coauthored a book on nuclear physics with Fermi, who was noted for building the world's first nuclear reactor. In 1954, after earning his PhD in physics, and with a recommendation from Fermi, Rosenfeld accepted a position as a teaching physicist at the University of California, Berkeley. At Berkeley, Rosenfeld joined the University of California Department of Physics and the particle physics research group at Lawrence Berkeley National Lab (LBNL) led by Nobel Laureate Luis Alvarez. Alvarez went on to win the Nobel Prize with research backed by his team of scientists that included Rosenfeld. In\nDocument [11](Title: Japanese Americans) J. Pedersen won the 1987 Nobel Prize in Chemistry for his methods of synthesizing crown ethers. Yoichiro Nambu won the 2008 Nobel Prize in Physics for his work on quantum chromodynamics and spontaneous symmetry breaking. Shuji Nakamura won the 2014 Nobel Prize in Physics for the invention of efficient blue light-emitting diodes. Michio Kaku is a theoretical physicist specializing in string field theory, and a well-known science popularizer. Ellison Onizuka became the first Asian American astronaut and was the mission specialist aboard \"Challenger\" at the time of its explosion. Immunologist Santa J. Ono became the first Japanese American president of\nDocument [12](Title: Nobel Prize in Physics) receive a diploma, a medal and a document confirming the prize amount. Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was\nDocument [13](Title: Paris in the Belle E\u0301poque) studying the fluorescence of uranium salts, discovered radioactivity in 1896, and in 1903 was awarded the Nobel Prize in physics for his discovery. Pierre Curie (1859-1906) and Marie Curie (1867-1934) jointly carried on Becquerel's work, discovering radium and polonium (1898). They jointly received the Nobel Prize for physics in 1903. Marie Curie became the first female professor at the University of Paris and won the Nobel Prize for chemistry in 1911. She was the first woman to be buried in the Panth\u00e9on. The neon light was used for the first time in Paris on 3 December 1910 in the Grand\nDocument [14](Title: Richard Feynman) by Michael Gottlieb and Ralph Leighton (Robert Leighton's son), with support from Kip Thorne and other physicists. Richard Feynman Richard Phillips Feynman (; May 11, 1918 \u2013 February 15, 1988) was an American theoretical physicist, known for his work in the path integral formulation of quantum mechanics, the theory of quantum electrodynamics, and the physics of the superfluidity of supercooled liquid helium, as well as in particle physics for which he proposed the parton model. For his contributions to the development of quantum electrodynamics, Feynman, jointly with Julian Schwinger and Shin'ichir\u014d Tomonaga, received the Nobel Prize in Physics in 1965.\nDocument [15](Title: Marie Curie) the Nobel Prize in Physics, \"in recognition of the extraordinary services they have rendered by their joint researches on the radiation phenomena discovered by Professor Henri Becquerel.\" At first the committee had intended to honor only Pierre Curie and Henri Becquerel, but a committee member and advocate for women scientists, Swedish mathematician Magnus Goesta Mittag-Leffler, alerted Pierre to the situation, and after his complaint, Marie's name was added to the nomination. Marie Curie was the first woman to be awarded a Nobel Prize. Curie and her husband declined to go to Stockholm to receive the prize in person; they were\nDocument [16](Title: E. C. George Sudarshan) had developed the breakthrough. In 2007, Sudarshan told the \"Hindustan Times\", \"The 2005 Nobel prize for Physics was awarded for my work, but I wasn't the one to get it. Each one of the discoveries that this Nobel was given for work based on my research.\" Sudarshan also commented on not being selected for the 1979 Nobel, \"Steven Weinberg, Sheldon Glashow and Abdus Salam built on work I had done as a 26-year-old student. If you give a prize for a building, shouldn\u2019t the fellow who built the first floor be given the prize before those who built the second\nDocument [17](Title: Rosalyn Sussman Yalow) the National Medal of Science, which is given to American individuals who deserve the highest honor in science and technology. In 1993, Yalow was inducted into the National Women's Hall of Fame. Rosalyn Sussman Yalow Rosalyn Sussman Yalow (July 19, 1921 \u2013 May 30, 2011) was an American medical physicist, and a co-winner of the 1977 Nobel Prize in Physiology or Medicine (together with Roger Guillemin and Andrew Schally) for development of the radioimmunoassay (RIA) technique. She was the second woman (the first being Gerty Cori), and the first American-born woman, to be awarded the Nobel Prize in physiology or\nDocument [18](Title: Poles) and chemist who conducted pioneering research on radioactivity and was the first woman to win a Nobel Prize, the first person and only woman to win twice, the only person to win twice in multiple sciences, and was part of the Curie family legacy of five Nobel Prizes. Another notable Polish expatriate scientist was Ignacy Domeyko (1802\u201389), a geologist and mineralogist who lived and worked in South America, in Chile. Kazimierz Funk (1884\u20131967), whose name is commonly anglicized as \"Casimir Funk\", was a Polish biochemist, generally credited with being among the first to formulate (in 1912) the concept of vitamins,\nDocument [19](Title: Nobel Prize) His son, George Paget Thomson, received the same prize in 1937 for showing that they also have the properties of waves. William Henry Bragg and his son, William Lawrence Bragg, shared the Physics Prize in 1915 for inventing the X-ray spectrometer. Niels Bohr was awarded the Physics prize in 1922, as was his son, Aage Bohr, in 1975. Manne Siegbahn, who received the Physics Prize in 1924, was the father of Kai Siegbahn, who received the Physics Prize in 1981. Hans von Euler-Chelpin, who received the Chemistry Prize in 1929, was the father of Ulf von Euler, who was awarded\nDocument [20](Title: Charles Russell Bardeen) business before marrying, and was an active figure in the art world. After her death from cancer in 1921, Charles married Ruth Hames. His son, Dr. John Bardeen, became the only person to win the Nobel Prize in Physics twice, in 1956 and 1972. Bardeen died in Madison, Wisconsin in 1935, from pancreatic cancer. He was succeeded as Dean of the University of Wisconsin Medical School by Dr. William Shainline Middleton. Charles Russell Bardeen Charles Russell Bardeen (8 February 1871 \u2013 12 June 1935) was an American physician and anatomist and the first dean of the University of Wisconsin Medical\n\n"""
q = "who got the first nobel prize in physics"

In [41]:
template = """<s>[INST] <<SYS>>
3 experts are discussing the question with a panel discussion, trying to solve it step by step using only the provided documents, and make sure the result is correct and avoid penalty:<</SYS>>

{search_results}

Question: {question}
Answer: [/INST]"""

In [42]:
# https://github.com/holarissun/PanelGPT

template = """<s>[INST] <<SYS>>
3 experts are discussing the question with a panel discussion, trying to solve it step by step, and make sure the result is correct:<</SYS>>

{search_results}

Question: {question}
Answer: [/INST]"""

In [43]:
template_cot = """<s>[INST] <<SYS>>
Write a high-quality answer for the given question using only the provided search results (some of which might be irrelevant).
<</SYS>>

{search_results}

Q: Answer the following question by reasoning step-by-step.
{question}
Answer:[/INST]"""

In [44]:
print(template.format(search_results=docs, question=q))

<s>[INST] <<SYS>>
3 experts are discussing the question with a panel discussion, trying to solve it step by step, and make sure the result is correct:<</SYS>>

"Document [1](Title: List of Nobel laureates in Physics) The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen, of Germany, who received 150,782 SEK, which is equal to 7,731,004 SEK in December 2007.  John Bardeen is the only laureate to win the prize twice—in 1956 and 1972. Maria Skłodowska-Curie also won two Nobel Prizes, for physics in 1903 and chemistry in 1911. William Lawrence Bragg was, until October 2014, the youngest ever Nobel laureate; he won the prize in 1915 at the age of 25. Two women have won the prize: Curie and Maria Goeppert-Mayer (1963). As of 2017, the prize has been awarded
Document [2](Title: Asian Americans in science and technology) Prize in physics for discovery of the subatomic particle J/ψ. Subrahmanyan Chandrasekhar shared the 1983 Nobel Prize in Physics and had the Chandra X-

In [17]:
llm = InferenceClient(
        model=HF_HUB_MODELS["llama-2-7b"],
        token=HUGGINGFACEHUB_API_TOKEN,
    )

In [18]:
output = llm.text_generation(
        """ <s>[INST] <<SYS>>
Write a high-quality answer for the given question using only the provided search results (some of which might be irrelevant).
<</SYS>>

Document [1](Title: List of Nobel laureates in Physics) The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen, of Germany, who received 150,782 SEK, which is equal to 7,731,004 SEK in December 2007.  John Bardeen is the only laureate to win the prize twice—in 1956 and 1972. Maria Skłodowska-Curie also won two Nobel Prizes, for physics in 1903 and chemistry in 1911. William Lawrence Bragg was, until October 2014, the youngest ever Nobel laureate; he won the prize in 1915 at the age of 25. Two women have won the prize: Curie and Maria Goeppert-Mayer (1963). As of 2017, the prize has been awarded

Question: who got the first nobel prize in physics
Answer: [/INST]""",
        max_new_tokens=200,
        temperature=1,
    )

HfHubHTTPError: 502 Server Error: Bad Gateway for url: https://api-inference.huggingface.co/models/meta-llama/Llama-2-7b-chat-hf

In [ ]:
output

'  Based on the provided search results, the first Nobel Prize in Physics was awarded to Wilhelm Conrad Röntgen in 1901.'

In [32]:
docs = """Document [1](Title: The High Chaparral) All the exterior filming was done at Old Tucson Studios in Arizona and in the nearby Saguaro National Park, although in a few later episodes there was some filming in California and (in season 3) in the Coronado National Forest south of Tucson.  The interiors were generally filmed at the NBC television studios in Burbank, Los Angeles.\nDocument [2](Title: The High Chaparral) The High Chaparral The High Chaparral is an American Western-themed television series starring Leif Erickson and Cameron Mitchell, which aired on NBC from 1967 to 1971. The series, made by Xanadu Productions in association with NBC Productions, was created by David Dortort, who had previously created \"Bonanza\" for the network. The theme song was also written and conducted by \"Bonanza\" scorer David Rose, who also scored the two-hour pilot. The show revolves around \"Big John\" Cannon (Erickson), a rancher living in the dry desert of southern Arizona Territory, near the Mexican border in Apache Indian country in the 1870s. He\nDocument [3](Title: The High Chaparral) box with season one (28 episodes) on May 2, 2012. Season two released on January 28, 2013. Season three released on September 3, 2013, and the final season released January 26, 2014. \"The High Chaparral\" is also released in Sweden with each season in two boxes. In Australia, they have also released \"The High Chaparral\" in four boxes. The High Chaparral The High Chaparral is an American Western-themed television series starring Leif Erickson and Cameron Mitchell, which aired on NBC from 1967 to 1971. The series, made by Xanadu Productions in association with NBC Productions, was created by David Dortort,\nDocument [4](Title: High Chaparral Theme Park) on the NBC (National Broadcasting Company) television network, for four seasons, 1967-1971, starring Leif Erickson, Linda Cristal, Henry Darrow and Cameron Mitchell. It focused on the adventures and drama about the large but struggling cattle ranch \"The High Chaparral\" on the high desert scrublands of southern Arizona Territory near the Mexican border and conflicts with the Apache Indians and their famous chief Cochise and the local United States Cavalry / U.S. Army troops. It's the stories of newly arrived owner \"Big John\" Cannon (former Union Army captain in the recent American Civil War, 1861-1865), his brother Buck, who fought for\nDocument [5](Title: High Chaparral) High Chaparral High Chaparral (1 March 1999 \u2013 21 December 2014) was an Irish Thoroughbred racehorse and sire. In a career that lasted from September 2001 to October 2003, he ran 13 times and won 10 races. His win in the Racing Post Trophy made him one of the highest-rated two-year-olds of 2001. In 2002, he defeated Hawk Wing in The Derby and won the Breeders' Cup Turf. At four, he defeated Falbrav and Alamshar in the Irish Champion Stakes and won a second Breeders' Cup Turf. High Chaparral was a dark-coated bay horse with a white star. He stood\nDocument [6](Title: Henry Darrow) \"The Wonderful Ice Cream Suit\". This brought him to the attention of television producer David Dortort, who immediately recruited him for his television western series \"The High Chaparral\", casting him as Manolito Montoya. Making its debut on American television in September 1967 NBC, it went on to last four seasons and was screened around the world. While on the show, both he and series' lead Cameron Mitchell became household names as the breakout stars of the show. Darrow is the first Latino actor to portray Zorro on television. (Jos\u00e9 Su\u00e1rez played Zorro in a 1953 Spanish film.) He starred in\nDocument [7](Title: Chaparral High School (Colorado)) 8th in 5A) and 2014 (placed 7th in 5A). Usually their field shows consist of either pop songs or musical compositions for movies, such as the How to Train Your Dragon soundtrack. Chaparral High School's theatre department was named the number one high school theatre program in the southwest US by \"Stage Directions\" magazine in November 2011. Their production of Victor Hugo's \"Les Mis\u00e9rables\" was awarded a total of five Bobby G awards by the Denver Center of the Performing Arts in 2013, including Best Overall Production, Best Direction, Best Musical Direction, and Best Performance by an Actor in a\nDocument [8](Title: The High Chaparral) runs the ranch, called \"The High Chaparral\" (named for a local plant/brush), with his brother Buck (Mitchell) and John's son Billy Blue (known as \"Blue Boy\") (Mark Slade). Blue Boy's mother, Anna-lee Cannon (Joan Caulfield), is killed in the first episode by an attacking Apache Indian arrow. John then marries a beautiful Mexican woman named Victoria (Linda Cristal), 30 years his junior, the daughter of powerful neighboring Mexican rancher Don Sebasti\u00e1n Montoya (Frank Silvera). In what is initially a marriage of convenience, she soon appreciates his strength and character, falls in love with him and becomes very supportive. John's marriage\nDocument [9](Title: The High Chaparral) a serious Western tv series, their scenes provided what is popularly known as comedy- or \"comic relief\" to the show. The other characters were gradually marginalized. Cattle ranching almost never featured in the storylines, which, whenever Mexican bandit or Indian troubles were not imminent, were much more likely to revolve around personal issues of drama with Manolito or Buck and some form of hell-raising - gambling, fighting, women, or whiskey (or a combination of them). Mark Slade (Blue Boy) did not appear in the final season, and Frank Silvera (Don Sebastian Montoya) had died while the series was still in\nDocument [10](Title: Mark Slade) name of the patriarch in \"The High Chaparral\". Mark Slade Mark Van Blarcom Slade (born May 1, 1939) is an American actor, artist, and author, particularly remembered for his role of Billy Blue Cannon on the NBC Western television series, \"The High Chaparral\". Born in Salem, Massachusetts, Slade is the son of Elinor (n\u00e9e Van Blarcom) and William A. Slade Jr., a Boston businessman and watercolor artist. Along with his two sisters and a brother, he grew up in the Danvers/Hamilton area of the North Shore. His parents divorced when he was 13, and his stepfather, Esmond R. Crowley Jr.,\nDocument [11](Title: High Chaparral Theme Park) High Chaparral Theme Park High Chaparral is a Wild West theme park and post-industrial museum of cultural artifacts and collectibles, located close to V\u00e4rnamo, Sweden. It opened in 1966 and was founded by Bengt Erlandson, more commonly known as Big Bengt. The park is located near Hillerstorp in Gnosj\u00f6 Municipality and is close to E4 motorway and Swedish national road 27. Big Bengt was born in 1922 in Br\u00e4nnehylte, Sm\u00e5land of the Kingdom of Sweden in Scandinavia, northern Europe. His parents owned a forest farm and a wood mill. Big Bengt was involved in starting over 1,000 companies. His interest\nDocument [12](Title: Palomar Mountain) friendly \"Space Brother\" from Venus named Orthon. The 1977 film \"The Crater Lake Monster\" had many scenes filmed on Palomar Mountain, including scenes shot at the summit restaurant, but not the scenes of the monster in a lake. High Point, is in the Cleveland National Forest, in the Palomar Mountain range is one of the highest peaks in San Diego County, at , surpassed by Cuyamaca Peak (at ) and Hot Springs Mountain (the county's highest point, at ). They are dwarfed by the higher San Bernardino Mountains a relatively short distance to the north, in San Bernardino County and\nDocument [13](Title: High Chaparral) at Fethard Equine Hospital during exploratory colic surgery due to a perforated intestine\". \"c = colt, f = filly, g = gelding\" High Chaparral High Chaparral (1 March 1999 \u2013 21 December 2014) was an Irish Thoroughbred racehorse and sire. In a career that lasted from September 2001 to October 2003, he ran 13 times and won 10 races. His win in the Racing Post Trophy made him one of the highest-rated two-year-olds of 2001. In 2002, he defeated Hawk Wing in The Derby and won the Breeders' Cup Turf. At four, he defeated Falbrav and Alamshar in the Irish\nDocument [14](Title: The High Chaparral) to Victoria also brings her brother Manolito (Henry Darrow) to live with the American \"gringo\" family on the extensive ranch. Among the many frequent guest stars were Rico Alaniz, Richard Bradford, Scott Brady, Rory Calhoun. Anthony Caruso, Chief Dan George, Roberto Contreras, Dennis Cross, Jim Davis, John Dehner, Bruce Dern, Charles Durning, Paul Fix, Steve Forrest, Ron Foster, Frank Gorshin, Ron Hagerthy, Ron Hayes, Myron Healey, Barbara Hershey, Don Keefer, Dan Kemp, Robert Loggia, Jack Lord, Tyler McVey, Ricardo Montalban, Joanna Moore, Nehemiah Persoff, Stuart Randall, Gilbert Roland, Ned Romero, Kurt Russell, Frank Silvera, Barry Sullivan, William Sylvester, William Tannen,\nDocument [15](Title: Chaparral High School (Colorado)) Chaparral High School (Colorado) Chaparral High School is a public high school located in Parker, Colorado, United States. It is a part of the Douglas County School District RE-1. The school is notable in Colorado for how it came to have an automated external defibrillator (AED) unit. Chaparral student Cody Schmidt died in 2003 after his heart stopped while in gym class, spurring his parents to raise funds to acquire AED units for all Douglas County schools. The school has the first rap club in Douglas County. Chaparral High School offers over thirty activity organizations, the majority of which are\nDocument [16](Title: Chaparral, New Mexico) in the neighboring city of El Paso, Texas. Several highways (state and federal) pass through Chaparral: Chaparral, New Mexico Chaparral is an unincorporated community and census-designated place (CDP) in Do\u00f1a Ana and Otero counties, New Mexico, United States. The population was 14,631 at the 2010 census. Chaparral is primarily a bedroom community for the neighboring city of El Paso, Texas, and the neighboring military installations of White Sands Missile Range and Fort Bliss. It is officially part of the Las Cruces Metropolitan Statistical Area. Chaparral is located in the southeast corner of Do\u00f1a Ana County and the southwest corner of\nDocument [17](Title: Dan Kemp) as well as \"High Chaparral\". In 1970, he appeared as a police lieutenant in the episode \"Beware the Wiles of the Stranger\" on Raymond Burr's NBC series \"Ironside\". He appeared twice in 1970 in different roles on the NBC western \"The High Chaparral\". In 1970, he also played the role of Vittorio in the film \"Cry Blood, Apache\". On November 30, 1970, he played the outlaw Lucas McCabe in the episode \"McCabe\" of CBS's \"Gunsmoke.\" Mitch Vogel played his teenage son, Dobie, who, because of hatred over the father having deserted him and his late mother, turns McCabe over to\nDocument [18](Title: High Chaparral) just over 16 hands high and was bred in County Tipperary, Ireland, by Sean Coughlan. In September 2000, he was consigned by the Mountain View Stud to the Tattersalls sales, where he was bought for 270,000 guineas by Dermot \"Demi\" O'Byrne on behalf of Michael Tabor's Coolmore organisation. He was sent into training with Aidan O'Brien at Ballydoyle. High Chaparral failed to make a winning debut, beaten by a short-head by Hot Trotter in a maiden at Punchestown on 30 September. A week later, he recorded his first victory when winning a maiden race at Tipperary by two and a\nDocument [19](Title: Mark Slade) in the Army\". In 1966, at the age of 27, he obtained one of his most memorable parts, as Billy Blue Cannon, the blond-haired, blue-eyed son of the ranch patriarch, John Cannon (Leif Erickson) on \"The High Chaparral\", set in the Arizona Territory. The series aired for four seasons. He went on to play Taylor Reed in the 1973 film \"Salty\" and reprised his role in the syndicated adventure series \"Salty\" (1974-1975). Slade's acting career continued into the early 1990s. He made more than 300 appearances on stage, screen, and television. Slade won international recognition and numerous awards for his\nDocument [20](Title: Chaparral, New Mexico) Chaparral, New Mexico Chaparral is an unincorporated community and census-designated place (CDP) in Do\u00f1a Ana and Otero counties, New Mexico, United States. The population was 14,631 at the 2010 census. Chaparral is primarily a bedroom community for the neighboring city of El Paso, Texas, and the neighboring military installations of White Sands Missile Range and Fort Bliss. It is officially part of the Las Cruces Metropolitan Statistical Area. Chaparral is located in the southeast corner of Do\u00f1a Ana County and the southwest corner of Otero County at (32.039072, \u2212106.429630). Its southern border is the Texas state line, the northern limit"""

q = "where was the tv show high chaparral filmed"
answers = ["California", "Saguaro National Park", "Old Tucson Studios in Arizona"]

In [33]:
output = llm.text_generation(
        template.format(search_results=docs, question=q),
        max_new_tokens=500,
        temperature=1,
    )
output = llm.text_generation(
        template.format(search_results=docs, question=q),
        max_new_tokens=500,
        temperature=1,
    )

In [34]:
output

'  The exterior filming of the TV show "High Chaparral" was done at Old Tucson Studios in Arizona and in the nearby Saguaro National Park, while the interior scenes were filmed at NBC television studios in Burbank, Los Angeles.'